In [2]:
# Dataset from: https://www.kaggle.com/litzar/fruits-classification

from keras.preprocessing import image

In [6]:
img = image.load_img('/fruits_360/Training/Apple Braeburn/0_100.jpg', target_size=(224, 224))

ImportError: Could not import PIL.Image. The use of `array_to_img` requires PIL.